In [3]:
# import imports
# import importlib
# importlib.reload(imports)
# from imports import *
import torch.nn as nn
import dill
import torch
from storage import Storage

class LSTM(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()

        self.lstm = nn.LSTM(INTERNAL_EMBEDDING_SIZE2, INTERNAL_EMBEDDING_SIZE2 // 2, 
                            num_layers=1, batch_first=True, bidirectional=True)
    def forward(self, x):
        return self.lstm(x)[0]


prefix = "results writers big"
with open(f"{prefix}/some_model_CLASS.dill", "rb") as file:
    Model = dill.load(file)
model = torch.load(f"{prefix}/some_model.pt", map_location=torch.device('cpu')) #).v1", map_location=torch.device('cuda:0'))

with open(f"{prefix}/storage_path.dill", "rb") as file:
    storage_path = dill.load(file)
with open(f"{prefix}/is_infected_test.dill", "rb") as file:
    is_infected = dill.load(file)

storage = Storage(str(storage_path))
params = storage.get_meta("params")

x_test = storage.get_meta("x_test").float()
y_test = storage.get_meta("y_test").float()
text_res = storage.get_meta("text_res_test")
is_infected = storage.get_meta("is_infected_test")

ModuleNotFoundError: No module named 'xformers'

In [4]:
dummy_input = x_test[0:1]
with torch.no_grad():
        print(model(dummy_input))
        # torch.onnx.export(model, dummy_input, "model.v1.onnx", verbose=True)

tensor([[ 7.9433, -7.6244, -3.0897]])


In [7]:
torch_script_graph, unconvertible_ops = torch.onnx.utils.unconvertible_ops(
    model, dummy_input #, opset_version=opset_version
)
unconvertible_ops

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



[]

[]

In [9]:
torch.onnx.export(model, (dummy_input, ), f"{prefix}/model.onnx", 
                input_names = ['input'],
                output_names = ['output'])#, verbose=True)
        # print(torch.onnx.export_to_pretty_string(model, (dummy_input, )))#, verbose=True))


================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [17]:
from inference import torch_model_runner, onnx_model_runner
for a in [dummy_input, np.zeros_like(dummy_input)]:
    print('torch:', torch_model_runner(model)(torch.Tensor(a)))
    print('onnx:', onnx_model_runner(f'{prefix}/model.onnx')(a))

torch: tensor([[ 7.9433, -7.6244, -3.0897]])
onnx: [[ 7.943265  -7.6243734 -3.0896897]]
torch: tensor([[ 5.6224, -6.0555, -3.0875]])
onnx: [[ 5.6224437 -6.0554595 -3.0875366]]


In [18]:
from inference import torch_model_runner, onnx_model_runner, infer

infer(params, onnx_model_runner(f"{prefix}/model.onnx"), 
      "Отмечается что 44-летняя россиянка первый человек которому предъявят обвинение в попытке вмешаться в выборы 2018 года По информации Минюста она и еще несколько человек выдавала себя за американскую политическую активистку и скрывала свое российское происхождение")
# print(infer(torch_model_runner(model), "Я пришел домой Мама сварила суп"))
# https://onnxruntime.ai/docs/reference/ort-format-models.html

# infer(onnx_model_runner("model.v1.onnx"), "Я пришел домой Мама сварила суп")


'Отмечается, что 44-летняя россиянка, первый человек, которому предъявят обвинение в попытке вмешаться в выборы 2018 года. По информации Минюста она и еще несколько человек выдавала себя за американскую политическую активистку и скрывала свое российское происхождение.'

In [21]:
infer(params, onnx_model_runner(f"{prefix}/model.onnx"), 
      text)


'Мама обычно холодная, как оконное стекло, была в такой ярости, что переколотила дома всю посуду. Хотела даже просить расчёт, но опомнилась: в других ресторанах и своих певиц хватало. Потому мама просто перестала брать Саню с собой, и он впервые оказался предоставлен самому себе. Поначалу Саня робел, один выйти наружу, скучал, слонялся, не зная, чем заняться, — то по комнатам, то по двору. Но к весне осмелел настолько, что исследовал сперва окрестные улицы, а потом и весь обитаемый воронежский мир. Мир этот оказался полон женщинами. Саня, разинув от восхищения рот, смотрел на проплывающих по Большой Дворянской разодетых дам, похожих на вазы с фруктами и цветами. На гимназисток, вечно державшихся дрожащими стайками, словно мотыльки. На горничных, девушек, спешащих по хозяйкиным делам и прошивающих город мелкими аккуратными стежками. Саня влюбился во всех них сразу — и навсегда.'

In [49]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse("фывфывфы")[0].inflect({'gent'})

Parse(word='фывфывфов', tag=OpencorporaTag('NOUN,inan,masc plur,gent'), normal_form='фывфывф', score=1.0, methods_stack=((FakeDictionary(), 'фывфывфов', 34, 7), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'фы')))

In [11]:
# text = """
# Кристина Герасимова (к концу десятилетия, кажется, поменявшая фамилию на Луна) к концу 2010-х оказалась одним из самых работоспособных музыкантов в русской поп-музыке (впрочем, и не только поп), выйдя на уровень, при котором она смогла себе позволить выпускать по альбому в год без потерь в качестве. Лучший из них, впрочем, «Заколдованные сны», как будто бы нарушал классическую для нее схему и уходил то в чистый и практически бессловесный электропоп, то в ностальгию по неприхотливой русской поп-музыке нулевых, то в постпанк — при этом всегда оставаясь песнями самой Луны.
# """
text = """

Мама, обычно холодная, как оконное стекло, была в такой ярости, что переколотила дома всю посуду. Хотела даже просить расчёт, но опомнилась: в других ресторанах и своих певиц хватало. Потому мама просто перестала брать Саню с собой, и он впервые оказался предоставлен самому себе. Поначалу Саня робел один выйти наружу, скучал, слонялся, не зная, чем заняться, — то по комнатам, то по двору. Но к весне осмелел настолько, что исследовал сперва окрестные улицы, а потом и весь обитаемый воронежский мир.

Мир этот оказался полон женщинами.

Саня, разинув от восхищения рот, смотрел на проплывающих по Большой Дворянской разодетых дам, похожих на вазы с фруктами и цветами. На гимназисток, вечно державшихся дрожащими стайками, словно мотыльки. На горничных девушек, спешащих по хозяйкиным делам и прошивающих город мелкими аккуратными стежками.

Саня влюбился во всех них сразу — и навсегда."""
#text = text.replace("\n", " ")
# print(text)

# text = text.replace(".", "").replace(",", "").replace("\n", " ")
# infer(params, onnx_model_runner(f"{prefix}/model.onnx"), text)

In [12]:
from razdel import tokenize
list(map(lambda i: i.text, tokenize(text)))

['Мама',
 ',',
 'обычно',
 'холодная',
 ',',
 'как',
 'оконное',
 'стекло',
 ',',
 'была',
 'в',
 'такой',
 'ярости',
 ',',
 'что',
 'переколотила',
 'дома',
 'всю',
 'посуду',
 '.',
 'Хотела',
 'даже',
 'просить',
 'расчёт',
 ',',
 'но',
 'опомнилась',
 ':',
 'в',
 'других',
 'ресторанах',
 'и',
 'своих',
 'певиц',
 'хватало',
 '.',
 'Потому',
 'мама',
 'просто',
 'перестала',
 'брать',
 'Саню',
 'с',
 'собой',
 ',',
 'и',
 'он',
 'впервые',
 'оказался',
 'предоставлен',
 'самому',
 'себе',
 '.',
 'Поначалу',
 'Саня',
 'робел',
 'один',
 'выйти',
 'наружу',
 ',',
 'скучал',
 ',',
 'слонялся',
 ',',
 'не',
 'зная',
 ',',
 'чем',
 'заняться',
 ',',
 '—',
 'то',
 'по',
 'комнатам',
 ',',
 'то',
 'по',
 'двору',
 '.',
 'Но',
 'к',
 'весне',
 'осмелел',
 'настолько',
 ',',
 'что',
 'исследовал',
 'сперва',
 'окрестные',
 'улицы',
 ',',
 'а',
 'потом',
 'и',
 'весь',
 'обитаемый',
 'воронежский',
 'мир',
 '.',
 'Мир',
 'этот',
 'оказался',
 'полон',
 'женщинами',
 '.',
 'Саня',
 ',',
 'рази

NameError: name 'model' is not defined

In [55]:
from time import time

start = time()
reps = 100
for i in range(0, reps):
    infer(onnx_model_runner("model.v1.onnx"), text)
end = time()

print(1 / ((end - start) / reps / len(text.split(' '))) / 8, 'tokens per second')



172.7963150459843 tokens per second


In [13]:
from time import time
dummy_input = torch.ones((1, params['INPUT_WORDS_CNT'], params['TOTAL_WORD_FEATURES_CNT']))
model.eval()

start = time()
reps = 10000
for i in range(0, reps):
    with torch.no_grad():
        model(dummy_input)
end = time()

print(1 / ((end - start) / reps) / 8, 'tokens per second')

129.86710276061967 tokens per second


In [14]:
from utils import count_parameters
count_parameters(model)

0.19739913940429688

In [3]:
import torch
model = torch.load("model.v1")
model

AttributeError: Can't get attribute 'Model' on <module '__main__'>

In [57]:
import numpy as np
import random
import pymorphy3
import numpy as np
import math
import pickle
from razdel import tokenize

from dataset_builder import calculate_word_features_for_tokens, PAD_TOKEN,get_word_features
from inference import torch_model_runner, onnx_model_runner, infer

onnx_model = onnx_model_runner("results writers big/model.onnx")
with open("params.pickle", "rb") as f:
    params = pickle.load(f)

class jsinfer:
    async def infer(arr):
        class wrapper:
            def to_py():
                return onnx_model(arr)
        return wrapper


async def infer(text):
    assert params["RETAIN_LEFT_PUNCT"] #

    #unpadded_tokens = text.split(' ')
    unpadded_tokens = list(map(lambda i: i.text, tokenize(text)))
    unpadded_tokens = list(filter(lambda x: len(x) > 0, unpadded_tokens))
    tokens = [PAD_TOKEN] * params['INPUT_WORDS_CNT_LEFT'] + unpadded_tokens + [PAD_TOKEN] * (params["INPUT_WORDS_CNT_RIGHT"] + 1)
    features = calculate_word_features_for_tokens(tokens, params)

    res = ""

    i = params['INPUT_WORDS_CNT_LEFT']
    while i < len(tokens) - params['INPUT_WORDS_CNT_RIGHT']:
        tokens_for_batch = tokens[i - params['INPUT_WORDS_CNT_LEFT']: i + params['INPUT_WORDS_CNT_RIGHT']]

        tokens_for_batch_copy = tokens_for_batch.copy()
        tokens_for_batch_copy.insert(params['INPUT_WORDS_CNT_LEFT'], '?')
        # print(" ".join(tokens_for_batch_copy))


        features_for_batch = features[i - params['INPUT_WORDS_CNT_LEFT']: i + params['INPUT_WORDS_CNT_RIGHT']]
        features_for_batch = np.stack((features_for_batch, ))
        arr = np.ascontiguousarray(features_for_batch, dtype=np.float32)
        output_probas = np.array((await jsinfer.infer(arr)).to_py())
        punct_idx = np.argmax(output_probas).item()
        punct = params["ID_TO_PUNCTUATION"][punct_idx]

        #tokens_for_batch_copy[params['INPUT_WORDS_CNT_LEFT']] =  f'<{tokens[i-1], punct, tokens[i]}>'
        tokens_for_batch_copy[params['INPUT_WORDS_CNT_LEFT']] =  f'<{punct}>'
        print(" ".join(tokens_for_batch_copy))

        # print(punct)
        # punct = '.'

        def add_i_token_to_res(i):
            nonlocal res
            if tokens[i] != 'PAD':
                if tokens[i] in '.,:!?':
                    res += tokens[i]
                else:
                    res += " " + tokens[i]

        def insert_punct(punct):
            nonlocal features
            tokens.insert(i, punct)
            features = np.concatenate((features[:i],
                    np.stack((get_word_features(punct, params), )),
                    features[i:]), 0)

        def replace_i_token_punct(punct):
            nonlocal features
            tokens[i] = punct
            features[i] = get_word_features(punct, params)

        def erase_i_token_punct():
            nonlocal features
            del tokens[i]
            features = np.concatenate((features[:i], features[i + 1:]), 0)

        if tokens[i] in '.,!?':
            if tokens[i] == punct or tokens[i] in "!?":# and punct in "."):
                add_i_token_to_res(i)
                add_i_token_to_res(i + 1)
                i += 2
            else:
                if punct != "$empty":
                    res += punct
                    replace_i_token_punct(punct)
                    i += 2
                else:
                    # res += " "
                    erase_i_token_punct()
                    # i += 0

        else:
            if punct != "$empty":
                res += punct #"{ins: " + punct + "`" + tokens[i] + "`}"
                add_i_token_to_res(i)
                insert_punct(punct)
                i += 2
            else:
                add_i_token_to_res(i)
                i += 1

    return res.strip()

await infer("Тест. тест. Тест. - Тест тест тест")

PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD <$empty> Привет ! Как дела ? Все хорошо . PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет <.> ! Как дела ? Все хорошо . PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет ! Как <$empty> дела ? Все хорошо . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет ! Как дела <.> ? Все хорошо . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет ! Как дела ? Все <$empty> хорошо . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD Привет ! Как дела ? Все хорошо <.> . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD


'Привет! Как дела? Все хорошо.'

In [7]:
from razdel import tokenize
from stream import Stream
a = [i for i in tokenize("Тест. тест. Тест. - Тест тест тест")]
a

[Substring(0, 4, 'Тест'),
 Substring(4, 5, '.'),
 Substring(6, 10, 'тест'),
 Substring(10, 11, '.'),
 Substring(12, 16, 'Тест'),
 Substring(16, 17, '.'),
 Substring(18, 19, '-'),
 Substring(20, 24, 'Тест'),
 Substring(25, 29, 'тест'),
 Substring(30, 34, 'тест')]

In [62]:
import numpy as np
import random
import pymorphy3
import numpy as np
import math
import pickle
from razdel import tokenize

from dataset_builder import calculate_word_features_for_tokens, PAD_TOKEN,get_word_features
from inference import torch_model_runner, onnx_model_runner, infer

onnx_model = onnx_model_runner("results writers big/model.onnx")
with open("params.pickle", "rb") as f:
    params = pickle.load(f)

class jsinfer:
    async def infer(arr):
        class wrapper:
            def to_py():
                return onnx_model(arr)
        return wrapper

In [194]:
from stream import Stream
import functools
from collections import deque
import random
random.seed(42)

@functools.lru_cache(maxsize=128)
def get_word_features_cached(word):
    return get_word_features(word, params).numpy()

class Substr:
    def __init__(self, text):
        self.text = text
    def __repr__(self) -> str:
        return f"Substring(-1, -1, {self.text})"

def d_as_str(d):
  return "<" + " ".join(map(lambda text: text.text, d))+ ">"
    
async def infer_optimal(params, text): 
  # print("INFERCENC IS WIERD\n" * 10)
  res = []
  last_inserted_pos = 0
  def sink(token, log=False):
    nonlocal last_inserted_pos
    if token.text == "PAD": return
    if log: print('sink', token)
    if isinstance(token, Substr):
      res.append(token.text)
      if log: print("added1 ", f"`{token.text}`", token)
    else:
      if last_inserted_pos != token.start:
        res.append(text[last_inserted_pos: token.start])
        if log: print("added2 ", f"`{text[last_inserted_pos: token.start]}`", last_inserted_pos, token.start)
      last_inserted_pos = token.stop
      res.append(token.text)
      if log: print("added3 ", f"`{token.text}`", token)

  def skip(token, log=False):
    nonlocal last_inserted_pos
    last_inserted_pos = token.stop
    if log: print('skip', token)


  async def predict_on_tokens(window_left, window_right, return_probas):
    features = [get_word_features_cached(i.text) for i in Stream(window_left).chain(window_right)]
    features_for_batch = np.stack((features, ))
    arr = np.ascontiguousarray(features_for_batch, dtype=np.float32)
    output_probas = np.array((await jsinfer.infer(arr)).to_py())
    # output_probas[0][0] += 2.
    if return_probas:
      return params["ID_TO_PUNCTUATION"], output_probas 
    punct_idx = np.argmax(output_probas).item()
    punct = params["ID_TO_PUNCTUATION"][punct_idx]
    return punct


  window_left = deque()
  window_right = deque()
  log = False
  skip_next = False
  for i in Stream.repeat(Substr(PAD_TOKEN), params['INPUT_WORDS_CNT_LEFT']) \
      .chain(Stream(tokenize(text))) \
      .chain(Stream.repeat(Substr(PAD_TOKEN), params["INPUT_WORDS_CNT_RIGHT"])):
    window_right.append(i)
    if len(window_right) <= params["INPUT_WORDS_CNT_RIGHT"]:
        continue
    assert len(window_right) == params["INPUT_WORDS_CNT_RIGHT"] + 1

    next_ = window_right.popleft()
    sink(next_)
    window_left.append(next_)
    if len(window_left) < params['INPUT_WORDS_CNT_LEFT']: 
      continue

    assert len(window_left) == params["INPUT_WORDS_CNT_LEFT"]
    assert len(window_right) == params["INPUT_WORDS_CNT_RIGHT"]

    if skip_next:
      prediction = "$skip" 
    else:
      # params["ID_TO_PUNCTUATION"], output_probas
      prediction = await predict_on_tokens(window_left, window_right, return_probas=False) 


    #random.choice([" ", "."])
    if log: print(d_as_str(window_left).rjust(100), prediction.center(6), d_as_str(window_right))

    def is_replaceable_punct(punct):
      return punct in ',.'

    if prediction == "$skip":
      pass
    elif prediction != "$empty":
      if is_replaceable_punct(window_right[0].text):
        if window_right[0].text != prediction:
          window_right[0].text = prediction
      else:
        window_left.append(Substr(prediction))
        sink(window_left[-1])
    else:
      if is_replaceable_punct(window_right[0].text):
          skip(window_right.popleft())

    skip_next = is_replaceable_punct(window_right[0].text)

    while len(window_left) != params['INPUT_WORDS_CNT_LEFT'] - 1: 
      token = window_left.popleft()

    if log: print(d_as_str(window_left).rjust(100), "      ", d_as_str(window_right))

  for i in window_right:
    sink(i)
  ress = "".join(res)
  return ress

In [ ]:
from striprtf.striprtf import rtf_to_text

In [191]:

#text = "Тест,  тест    Тест. - Тест, тест, тест...."
# text = "Тест,  тест.    Тест. - Тест, тест, тест....."
text = """C++ (читается си-плюс-плюс[2][3]) — компилируемый, статически типизированный язык программирования общего назначения.

Поддерживает такие парадигмы программирования, как процедурное программирование, объектно-ориентированное программирование, обобщённое программирование. Язык имеет богатую стандартную библиотеку, которая включает в себя распространённые контейнеры и алгоритмы, ввод-вывод, регулярные выражения, поддержку многопоточности и другие возможности. C++ сочетает свойства как высокоуровневых, так и низкоуровневых языков[4][5]. В сравнении с его предшественником — языком C — наибольшее внимание уделено поддержке объектно-ориентированного и обобщённого программирования[5].

C++ широко используется для разработки программного обеспечения, являясь одним из самых популярных языков программирования[мнения 1][мнения 2]. Область его применения включает создание операционных систем, разнообразных прикладных программ, драйверов устройств, приложений для встраиваемых систем, высокопроизводительных серверов, а также компьютерных игр. Существует множество реализаций языка C++, как бесплатных, так и коммерческих и для различных платформ. Например, на платформе x86 это GCC, Visual C++, Intel C++ Compiler, Embarcadero (Borland) C++ Builder и другие. C++ оказал огромное влияние на другие языки программирования, в первую очередь на Java и C#.

Синтаксис C++ унаследован от языка C. Изначально одним из принципов разработки было сохранение совместимости с C. Тем не менее C++ не является в строгом смысле надмножеством C; множество программ, которые могут одинаково успешно транслироваться как компиляторами C, так и компиляторами C++, довольно велико, но не включает все возможные программы на C. """

# for i in range(10):
rres = await infer_optimal(params, text)
text, ress

INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD



('C++ (читается си-плюс-плюс[2][3]) — компилируемый, статически типизированный язык программирования общего назначения.\n\nПоддерживает такие парадигмы программирования, как процедурное программирование, объектно-ориентированное программирование, обобщённое программирование. Язык имеет богатую стандартную библиотеку, которая включает в себя распространённые контейнеры и алгоритмы, ввод-вывод, регулярные выражения, поддержку многопоточности и другие возможности. C++ сочетает свойства как высокоуровневых, так и низкоуровневых языков[4][5]. В сравнении с его предшественником — языком C — наибольшее внимание уделено поддержке объектно-ориентированного и обобщённого программирования[5].\n\nC++ широко используется для разработки программного обеспечения, являясь одним из самых популярных языков программирования[мнения 1][мнения 2]. Область его применения включает создание операционных систем, разнообразных прикладных программ, драйверов устройств, приложений для встраиваемых систем, высокопр

In [197]:
with open("../validation/text1.txt") as val:
    text = val.read()

# len(list(map(lambda x: x.text, tokenize(text))))
text_res = await infer_optimal(params, text.replace(".", "").replace(",", ""))

INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD
INFERCENC IS WIERD



In [ ]:
from striprtf.striprtf import rtf_to_text


In [198]:
import diff_match_patch as dmp_module
from collections import defaultdict, Counter

dmp = dmp_module.diff_match_patch()
diff = dmp.diff_main(text, text_res)

diff_aggregate = defaultdict(lambda : 0)
def sink2():
  global cur_add, cur_remove
  if cur_add == "" and cur_remove == "":
    return
  if cur_add == "":
    diff_aggregate['remove ' + cur_remove] += 1
    cur_remove = ""
    return
  if cur_remove == "":
    diff_aggregate['add ' + cur_add] += 1
    cur_add = ""
    return
   
  diff_aggregate['replace ' + cur_remove + " with " + cur_add] += 1
  cur_add = ""
  cur_remove = ""

cur_remove = ""
cur_add = ""

UNCHANGED = 0
ADD = 1
REMOVE = -1

for change in diff:
    if change[0] == UNCHANGED:
       c = Counter(change[1])
       diff_aggregate['unchanged .'] += c['.']
       diff_aggregate['unchanged ,'] += c[',']
       sink2() 
    elif change[0] == ADD:
      cur_add += change[1]
    elif change[0] == REMOVE:
      cur_remove += change[1]
    else:
       raise Exception("Unknown format")
    
sink2()

diff_aggregate

defaultdict(<function __main__.<lambda>()>,
            {'unchanged .': 455,
             'unchanged ,': 517,
             'add ,': 79,
             'remove ,': 158,
             'remove .': 24,
             'add .': 63,
             'replace , with .': 2})

In [ ]:
# С исходным текстом
defaultdict(<function __main__.<lambda>()>,
            {'unchanged .': 472,
             'unchanged ,': 634,
             'add ,': 180,
             'remove ,': 42,
             'add .': 86,
             'remove .': 7,
             'replace , with .': 1})

In [ ]:
# Без исходного текста
defaultdict(<function __main__.<lambda>()>,
            {'unchanged .': 466,
             'unchanged ,': 580,
             'add ,': 177,
             'remove ,': 95,
             'add .': 83,
             'remove .': 13,
             'replace , with .': 2})

In [ ]:
# тюнинг +2 с исходным текстом
defaultdict(<function __main__.<lambda>()>,
            {'unchanged .': 468,
             'unchanged ,': 594,
             'add ,': 78,
             'remove ,': 82,
             'add .': 69,
             'remove .': 11,
             'replace , with .': 1})

In [ ]:
#тюнинг +2 без исходного текста
defaultdict(<function __main__.<lambda>()>,
            {'unchanged .': 455,
             'unchanged ,': 517,
             'add ,': 79,
             'remove ,': 158,
             'remove .': 24,
             'add .': 63,
             'replace , with .': 2})

In [163]:
from collections import Counter
Counter(text)[',s'], Counter(text_res)[',']

(0, 757)

In [151]:
import diff_match_patch as dmp_module

one =   'beep boop'
other = 'beep boob blah'

dmp = dmp_module.diff_match_patch()
diff = dmp.diff_lineMode(one, other, False)

for change in diff:
  print(change[0], change[1])


0 beep boo
-1 p
1 b blah


In [146]:
change[1]

'beep boo'

In [139]:
text[8000:9000]

'овника… Жизнь грохочет там, где насилие стремится стереть ее своеобразие и особенности.\n\nВнимательный и небрежный глаз седого машиниста следил за мельканием бетонных столбиков, высоких мачт с вращающимися прожекторами, бетонированных башен, где в стеклянном фонаре виднелся охранник у пулемета. Машинист мигнул помощнику, паровоз дал предупредительный сигнал. Мелькнула освещенная электричеством будка, очередь машин у опущенного полосатого шлагбаума, бычий красный глаз светофора.\n\n(По В. Гроссману)\n6\nРябиновый дол\n\nЗатерялся в перелесках рябиновый дол. Рассказывают, что давным-давно здесь, в этом долу, молнией убило убогого странника. А богдановский пастух возьми и посади на том месте две рябинки на память. А теперь вон их сколько, не сосчитаешь: весь дол в них, по обоим склонам. Рябинки одна на другую не похожие, и ягоды тоже: то они розовые, то алые, то темно-красные.\n\nЯ люблю заходить сюда в августе, когда наливается рябина, исподволь краснеют ее ягоды и свешиваются тяжелыми

In [140]:
len(text), len(text_res)

(39849, 40066)

In [143]:
import diff_match_patch as dmp_module

one = 'beep boop'
other = 'beep boob blah'

dmp = dmp_module.diff_match_patch()
diff = dmp.diff_main(one, other)

for change in diff:
    print(change[0], " | ", change[1])

0  |  beep boo
-1  |  p
1  |  b blah


In [136]:
text_res[8781:]

'е сосчитаешь: весь дол в них, по обоим склонам. Рябинки одна на другую, не похожие и ягоды тоже: то они розовые, то алые, то темно-красные.\n\nЯ люблю заходить сюда в августе, когда наливается рябина, исподволь краснеют ее ягоды и свешиваются тяжелыми кистями. В сентябре залюбуешься резными листьями, багряными и красными от алых зорь, и рябинового огня. А в октябре, когда кругом стоят голые деревья, этот дол, еще долго радует глаз. Издали кажется, что алые кисти висят не на тонких ветвях, а прямо в воздухе.\n\nЭтот дол хорош, и зимой, даже еще лучше на белом поле. Потому и склоны его, такие нарядные и праздничные, что рябит в глазах от красного и белого, рябинового и снежного. И не поймешь, чего тут больше. Дол в конце делает полукруг, делают полукруг и рябинки. Вот и кажется, что эту снежную долину украсил щедрый волшебник рубиновым ожерельем. А это ожерелье, то покроется снегом, то опять останется голым, озябшим, постукивая на ветру, замороженными ягодами.\n\nКогда же падает сырой с

In [112]:
text

'1\nЗайчата\n\nЗайцы-беляки живут в теплом климате; приносят они от двух до пяти зайчат четыре-пять раз в год. В марте у зайчихи рождаются «настовички» (в это время еще лежит снег, покрытый твердой коркой – настом).\n\nЕсть разные мнения о родительской заботливости зайчихи. Одни ученые считают, что зайчиха – хорошая мамаша, она остается рядом с детишками, далеко от себя не отпускает их, обучает, а в случае опасности притворяется больной или раненой и отводит от зайчат хищника.\n\nБытует и другое мнение: зайчиха оставляет новорожденных где-нибудь под кустиком или в траве и убегает. Зайчата пока обеспечены едой: мать накормила их, и в желудке у каждого зайчонка есть запас очень жирного молока. Через какое-то время мамаша вернется и снова покормит зайчат, или покормит их другая зайчиха, пробегая мимо.\n\nЗайчата рождаются вполне развитыми, оформившимися, зрячими, они хорошо растут и через несколько дней начинают питаться самостоятельно.\n2\nБарсуки\n\nВесной, перед выходом из норы, барсук

In [132]:
from text_diff import text_differences

text_1 = [
    'Hello',
    'World',
    'How are you ?',
]

text_2 = [
    'Hello',
    'World!',
    'How are u ?',
]

diff = text_differences(text_1, text_2)
for line in diff.diff_lines:
    print(line)

UnchangedLine(content='Hello')
ModifiedLine(content_before='World', mask_before="     ", content_after='World!', mask_after="     +")
ModifiedLine(content_before='How are you ?', mask_before="        --", content_after='How are u ?', mask_after="          ")


In [134]:
d = difflib.Differ()
e = d.compare(text, text_res)        #set the compare output to a variable

for i in e:
    if i.startswith("-"):         #if that char start with "-" is not a match
        print(i + "index is different")


- ,index is different
- .index is different
- .index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
-  index is different
- сindex is different
- кindex is different
- оindex is different
- лindex is different
- ьindex is different
- кindex is different
- оindex is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
-  index is different
- оindex is different
- сindex is different
- аindex is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- .index is different
- .index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index is different
- ,index i

In [113]:
text_res

'1\nЗайчата\n\nЗайцы-беляки живут в теплом климате; приносят они от двух до пяти зайчат, четыре-пять раз в год. В марте у зайчихи рождаются «настовички» (в это время еще лежит снег, покрытый твердой коркой – настом).\n\nЕсть разные мнения о родительской заботливости, зайчихи. Одни ученые считают, что зайчиха – хорошая мамаша, она остается рядом с детишками, далеко от себя не отпускает их, обучает, а в случае опасности притворяется больной или раненой и отводит от зайчат, хищника.\n\nБытует и другое мнение: зайчиха оставляет новорожденных где-нибудь под кустиком, или в траве, и убегает. Зайчата пока обеспечены едой: мать накормила их, и в желудке у каждого зайчонка есть запас очень жирного молока. Через какое-то время мамаша вернется и снова покормит зайчат, или покормит их другая зайчиха, пробегая мимо.\n\nЗайчата рождаются вполне развитыми, оформившимися зрячими, они хорошо растут и через несколько дней начинают питаться самостоятельно.\n2\nБарсуки.\n\nВесной, перед выходом из норы, б

In [ ]:
Тест,  тест    Тест. - Тест, тест, тест....

In [ ]:
async def infer_optimal(params, text): 
  # print("INFERCENC IS WIERD\n" * 10)
  res = []
  last_inserted_pos = 0
  def sink(token, log=False):
    nonlocal last_inserted_pos
    if token.text == "PAD": return
    if log: print('sink', token)
    if isinstance(token, Substr):
      res.append(token.text)
      if log: print("added1 ", f"`{token.text}`", token)
    else:
      if last_inserted_pos != token.start:
        res.append(text[last_inserted_pos: token.start])
        if log: print("added2 ", f"`{text[last_inserted_pos: token.start]}`", last_inserted_pos, token.start)
      last_inserted_pos = token.stop
      res.append(token.text)
      if log: print("added3 ", f"`{token.text}`", token)

  def skip(token, log=False):
    nonlocal last_inserted_pos
    last_inserted_pos = token.stop
    if log: print('skip', token)

  def sink_remaining():
     res.append(text[last_inserted_pos:])


  async def predict_on_tokens(window_left, window_right, return_probas):
    features = [get_word_features_cached(i.text) for i in Stream(window_left).chain(window_right)]
    features_for_batch = np.stack((features, ))
    arr = np.ascontiguousarray(features_for_batch, dtype=np.float32)
    output_probas = np.array((await jsinfer.infer(arr)).to_py())
    # output_probas[0][0] += 2.
    if return_probas:
      return params["ID_TO_PUNCTUATION"], output_probas 
    punct_idx = np.argmax(output_probas).item()
    punct = params["ID_TO_PUNCTUATION"][punct_idx]
    return punct


  window_left = deque()
  window_right = deque()
  log = False
  skip_next = False
  for i in Stream.repeat(Substr(PAD_TOKEN), params['INPUT_WORDS_CNT_LEFT']) \
      .chain(Stream(tokenize(text))) \
      .chain(Stream.repeat(Substr(PAD_TOKEN), params["INPUT_WORDS_CNT_RIGHT"])):
    window_right.append(i)
    if len(window_right) <= params["INPUT_WORDS_CNT_RIGHT"]:
        continue
    assert len(window_right) == params["INPUT_WORDS_CNT_RIGHT"] + 1

    next_ = window_right.popleft()
    sink(next_)
    window_left.append(next_)
    if len(window_left) < params['INPUT_WORDS_CNT_LEFT']: 
      continue

    assert len(window_left) == params["INPUT_WORDS_CNT_LEFT"]
    assert len(window_right) == params["INPUT_WORDS_CNT_RIGHT"]

    if skip_next:
      prediction = "$skip" 
    else:
      # params["ID_TO_PUNCTUATION"], output_probas
      prediction = await predict_on_tokens(window_left, window_right, return_probas=False) 


    #random.choice([" ", "."])
    if log: print(d_as_str(window_left).rjust(100), prediction.center(6), d_as_str(window_right))

    def is_replaceable_punct(punct):
      return punct in ',.'

    if prediction == "$skip":
      pass
    elif prediction != "$empty":
      if is_replaceable_punct(window_right[0].text):
        if window_right[0].text != prediction:
          window_right[0].text = prediction
      else:
        window_left.append(Substr(prediction))
        sink(window_left[-1])
    else:
      if is_replaceable_punct(window_right[0].text):
          skip(window_right.popleft())

    skip_next = is_replaceable_punct(window_right[0].text)

    while len(window_left) != params['INPUT_WORDS_CNT_LEFT'] - 1: 
      token = window_left.popleft()

    if log: print(d_as_str(window_left).rjust(100), "      ", d_as_str(window_right))

  for i in window_right:
    sink(i)
  sink_remaining()
  ress = "".join(res)
  return ress


In [30]:
print(xformers.components.attention._USE_SPUTNIK)

False


In [22]:
import xformers
from xformers.components import attention
# xformers.components.attention._DENSITY_THRESHOLD = 100.
attention._USE_SPUTNIK = False


import dill
import torch
from storage import Storage
prefix = "results_big_model_GOOD_2022-05-15"
print("ADD RESULTS ")
with open(f"{prefix}/some_model_CLASS.dill", "rb") as file:
    Model = dill.load(file)
model = torch.load(f"{prefix}/some_model.pt", map_location=torch.device('cpu')) #).v1", map_location=torch.device('cuda:0'))

with open(f"{prefix}/storage_path.dill", "rb") as file:
    storage_path = dill.load(file)


storage = Storage(str(storage_path))
params = storage.get_meta("params")
x_test = storage.get("x", 0).float()
print(x_test.shape)

dummy_input = x_test[0:10]
print(dummy_input.shape)


from xformers.components.attention.attention_patterns import (
    local_1d_pattern)
from xformers.components.attention.core import scaled_dot_product_attention

for i in [3, 4, 5, 6, 7]:
    att = list(model.children())[0][i].wrap_att.layer.sublayer.children().__next__()
    # att._get_local_mask = lambda shape: local_1d_pattern(shape[1], att.window_size * 3).to_sparse()
    att.forward = lambda q, k, v, att_mask: scaled_dot_product_attention(
            q=q, k=k, v=v, 
            att_mask= local_1d_pattern(q.shape[1], att.window_size * 3).to_sparse() & att_mask, 
            dropout=att.attn_drop
        )
    att.attention_mask = None


# dataset_mem_reader =
with torch.no_grad():
    print(model(x_test))

ADD RESULTS 
torch.Size([1675, 32, 489])
torch.Size([10, 32, 489])
tensor([[-1.7760,  1.6977, -1.7477],
        [-1.8823,  1.5165, -1.8250],
        [ 0.6482, -1.0438, -3.3550],
        ...,
        [ 0.7262, -0.2323, -4.8659],
        [ 2.0915, -0.6271, -6.7218],
        [ 2.0524, -0.7290, -6.8905]])


In [23]:
with torch.no_grad():
    print(model(x_test))

tensor([[-1.7760,  1.6977, -1.7477],
        [-1.8823,  1.5165, -1.8250],
        [ 0.6482, -1.0438, -3.3550],
        ...,
        [ 0.7262, -0.2323, -4.8659],
        [ 2.0915, -0.6271, -6.7218],
        [ 2.0524, -0.7290, -6.8905]])


In [18]:
list(model.children())[0]

Sequential(
  (0): Linear(in_features=489, out_features=512, bias=True)
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): xFormerEncoderBlock(
    (pose_encoding): SinePositionalEmbedding()
    (wrap_att): Residual(
      (layer): PreNorm(
        (norm): FusedLayerNorm()
        (sublayer): MultiHeadDispatch(
          (attention): LocalAttention(
            (attn_drop): Dropout(p=0.1, inplace=False)
          )
          (in_proj_container): InputProjection(
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (resid_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=512, out_features=512, bias=True)
        )
      )
    )
    (wrap_ff): PostNorm(
      (norm): FusedLayerNorm()
      (sublayer): Residual(
      

In [19]:
list(model.children())[0][3][1][0][0]

TypeError: 'xFormerEncoderBlock' object is not subscriptable

LocalAttention(
  (attn_drop): Dropout(p=0.1, inplace=False)
)